# Library

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download Dataset

In [2]:
imdb, info = tfds.load("imdb_reviews",with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete44Q4OS/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete44Q4OS/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete44Q4OS/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data, test_data = imdb['train'], imdb['test']

# Preprocessing

## Change dataset tensor based to numpy array based

In [4]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

for sentences,labels in train_data:
    training_sentences.append(str(sentences.numpy()))
    training_labels.append(labels.numpy())

for sentences,labels in test_data:
    testing_sentences.append(str(sentences.numpy()))
    testing_labels.append(labels.numpy())

## Change labels from array to numpy array

In [5]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

# Models

## Setting Parameters

Vocabulary size refers to the total number of unique words used in the model. A larger vocabulary size results in a more complex model. Embedding dimension represents the vector dimension of each word, and a larger embedding dimension also increases model complexity. The truncation type determines the placement of padding in the sequence. When using "pre" for padding, the padding is added at the beginning of the sequence. OOV_tok is used to handle out-of-vocabulary words and assigns a zero string value to the first sequence.

In [6]:
vocab_size = 10000
embedding_dim=16
max_length = 120
trunc_type= 'pre'
oov_tok=""

In [7]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
word_index

{'': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'is': 7,
 'br': 8,
 'in': 9,
 'it': 10,
 'i': 11,
 'this': 12,
 'that': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 "'s": 21,
 'on': 22,
 'you': 23,
 'not': 24,
 'are': 25,
 'his': 26,
 'he': 27,
 'have': 28,
 'be': 29,
 'one': 30,
 'all': 31,
 'at': 32,
 'by': 33,
 'they': 34,
 'an': 35,
 'who': 36,
 'so': 37,
 'from': 38,
 'like': 39,
 'her': 40,
 "'t": 41,
 'or': 42,
 'just': 43,
 'there': 44,
 'about': 45,
 'out': 46,
 "'": 47,
 'has': 48,
 'if': 49,
 'some': 50,
 'what': 51,
 'good': 52,
 'more': 53,
 'very': 54,
 'when': 55,
 'she': 56,
 'up': 57,
 'can': 58,
 'b': 59,
 'time': 60,
 'no': 61,
 'even': 62,
 'my': 63,
 'would': 64,
 'which': 65,
 'story': 66,
 'only': 67,
 'really': 68,
 'see': 69,
 'their': 70,
 'had': 71,
 'were': 72,
 'me': 73,
 'well': 74,
 'we': 75,
 'than': 76,
 'much': 77,
 'been': 78,
 'get': 79,
 'bad': 80,
 'will': 81,
 'people': 82,
 'do': 83,
 'al

In [8]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating = trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

## Make a model with GRU

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9600      
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 169,665
Trainable params: 169,665
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
history=model.fit(padded, training_labels_final, epochs=10,validation_data = (testing_padded,testing_labels_final))


Epoch 1/10
782/782 [==============================] - 59s 63ms/step - loss: 0.4319 - accuracy: 0.7877 - val_loss: 0.3430 - val_accuracy: 0.8518
Epoch 2/10
782/782 [==============================] - 20s 25ms/step - loss: 0.2623 - accuracy: 0.8943 - val_loss: 0.3624 - val_accuracy: 0.8478
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2072 - accuracy: 0.9222 - val_loss: 0.3930 - val_accuracy: 0.8452
Epoch 4/10
782/782 [==============================] - 19s 24ms/step - loss: 0.1744 - accuracy: 0.9377 - val_loss: 0.4144 - val_accuracy: 0.8442
Epoch 5/10
782/782 [==============================] - 14s 17ms/step - loss: 0.1430 - accuracy: 0.9498 - val_loss: 0.4227 - val_accuracy: 0.8434
Epoch 6/10
782/782 [==============================] - 15s 20ms/step - loss: 0.1091 - accuracy: 0.9631 - val_loss: 0.4890 - val_accuracy: 0.8404
Epoch 7/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0740 - accuracy: 0.9760 - val_loss: 0.5707 - val_accuracy: